#Install the Dependencies to connect the backend server with frontend using FastAPI

In [ ]:
!pip install python-dotenv

In [ ]:
!pip install fastapi


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.4 MB/s eta 0:00:00


In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.6/109.6 kB 5.2 MB/s eta 0:00:00


In [ ]:
!pip install fastapi uvicorn python-dotenv google-colab groq


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 27.5 MB/s eta 0:00:00


In [ ]:
!pip install fastapi uvicorn pyngrok


#Set the GROQ API KEY

In [ ]:
import os
from google.colab import userdata
os.environ["GROQ_API_KEY"]= userdata.get('GROQ_API_KEY')

In [ ]:
from groq import Groq
client = Groq()

#Fetching the Data Using FastAPI through Post endpoint and Get Endpoint

In [ ]:
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from typing import Optional
import os
import openai

# Initialize FastAPI app
app = FastAPI()

# Add CORS middleware
origins = ["http://localhost:5173"]
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_methods=["*"],
    allow_headers=["*"],
)

# In-memory storage for revision data
class RevisionStore:
    def __init__(self):
        self.revision_data = None

    def set_revision(self, data: str):
        self.revision_data = data

    def get_revision(self) -> Optional[str]:
        return self.revision_data

revision_store = RevisionStore()

# Initialize OpenAI client
try:
    client = openai.OpenAI(
        base_url="https://api.groq.com/openai/v1",
        api_key=os.environ.get("GROQ_API_KEY")
    )
    if not client.api_key:
        raise ValueError("GROQ_API_KEY is not set in the environment variables")
except Exception as e:
    print(f"Error initializing OpenAI client: {e}")
    client = None

# Request model for /guide endpoint
class GuideRequest(BaseModel):
    subject: str
    qualification: str
    additional_context: Optional[str] = None

@app.post("/guide")
async def generate_guide(request: GuideRequest):
    try:
        if not client:
            raise HTTPException(status_code=500, detail="OpenAI client is not initialized")

        # Construct prompt
        prompt = (
            f"Generate an array of guide sentences like formulas or ideas for the "
            f"{request.subject} subject for {request.qualification}. "
        )
        if request.additional_context:
            prompt += f"Additional context: {request.additional_context}. "

        prompt += (
            f"Give it to me in the form of an array of points, and it must contain 5-8 points. "
            f"Only provide me the array; don't provide me any other information."
        )

        # Generate completion using OpenAI client
        response = client.completion(
            model="llama2-70b-versatile",
            prompt=prompt,
            temperature=0.7,
            max_tokens=150,
            top_p=1
        )

        # Extract response content
        generated_content = response["choices"][0]["text"].strip()  # Adjust key names based on actual API response

        # Save revision data
        revision_store.set_revision(generated_content)

        return {
            "message": "Revision data updated successfully",
            "revision": revision_store.get_revision(),
        }

    except Exception as e:
        print(f"Error in /guide: {e}")
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/getGuide")
async def get_guide():
    try:
        revision_data = revision_store.get_revision()
        if not revision_data:
            raise HTTPException(status_code=404, detail="No revision data found")

        return {"revision": revision_data}
    except Exception as e:
        print(f"Error in /getGuide: {e}")
        raise HTTPException(status_code=500, detail="Internal Server Error")

@app.get("/")
def root():
    return {"message": "Server is running!"}


#Unit Testing using Ngrok

In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("2rXjCkazzN8GJQAhSXT8yMHG4TG_7AJwN49Exh4kTz5DanXmK")
# List all active tunnels
tunnels = ngrok.get_tunnels()
print("Active tunnels:")
for tunnel in tunnels:
    print(tunnel)

# Disconnect all tunnels
for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)
print("All tunnels disconnected.")

# Now you can try to connect again
public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Active tunnels:
NgrokTunnel: "https://476c-34-19-111-129.ngrok-free.app" -> "http://localhost:8000"
All tunnels disconnected.
Public URL: NgrokTunnel: "https://6b80-34-19-111-129.ngrok-free.app" -> "http://localhost:8000"


#Send the Post request to the server using Payload

In [ ]:
import requests

url ="https://6b80-34-19-111-129.ngrok-free.app/guide"  # Replace with your ngrok public URL
payload = {
    "subject": "Mathematics",
    "qualification": "High School",
    "additional_context": "Focus on algebra and geometry."
}

response = requests.post(url, json=payload)
print(response.json())


Error in /guide: 'Groq' object has no attribute 'chat_completion'
INFO:     34.19.111.129:0 - "POST /guide HTTP/1.1" 500 Internal Server Error
{'detail': 'Internal Server Error'}


In [ ]:
pip show groq

Name: groq
Version: 0.15.0
Summary: The official Python library for the groq API
Home-page: https://github.com/groq/groq-python
Author: 
Author-email: Groq <support@groq.com>
License: 
Location: /usr/local/lib/python3.10/dist-packages
Requires: anyio, distro, httpx, pydantic, sniffio, typing-extensions
Required-by: 
